# Image Classification with Keras
Skin diseases detection.
Using OpenCV, Tensorflow-Keras

### import necessary library


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16


import cv2
import os

In [6]:
!kaggle datasets download -d shubhamgoel27/dermnet

dermnet.zip: Skipping, found more recently modified local copy (use --force to force download)


In [112]:
diseases = os.listdir("/content/train/")
len(diseases)

23

## Labelling image

In [100]:
def labellingImg(path):
  # vectorize the data
  X = []
  y = []

  x_path = os.listdir(path)
  num_classes = len(x_path)
  y_label = 0
  for i in x_path:
    img_path = path + i
    file_path = os.listdir(img_path)
    # print(i)
    for j in file_path:
      img_file = img_path + '/' + j

      if img_file[-3:] == 'jpg':
        # print(img_file[-3:])
        img = cv2.imread(img_file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(180,180))
      else:
        img = None

      if img is not None:
        img = tf.constant(img)
        X.append(img)
        y.append(y_label)
    print('Y: ' + i )
    y_label += 1
  y = keras.utils.to_categorical(y, num_classes)

  return X, y

In [101]:
X_train, y_train = labellingImg('/content/train/')

Y: Scabies Lyme Disease and other Infestations and Bites
Y: Eczema Photos
Y: Warts Molluscum and other Viral Infections
Y: Nail Fungus and other Nail Disease
Y: Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
Y: Hair Loss Photos Alopecia and other Hair Diseases
Y: Bullous Disease Photos
Y: Vasculitis Photos
Y: Exanthems and Drug Eruptions
Y: Lupus and other Connective Tissue diseases
Y: Psoriasis pictures Lichen Planus and related diseases
Y: Urticaria Hives
Y: Poison Ivy Photos and other Contact Dermatitis
Y: Vascular Tumors
Y: Systemic Disease
Y: Melanoma Skin Cancer Nevi and Moles
Y: Herpes HPV and other STDs Photos
Y: Acne and Rosacea Photos
Y: Light Diseases and Disorders of Pigmentation
Y: Tinea Ringworm Candidiasis and other Fungal Infections
Y: Cellulitis Impetigo and other Bacterial Infections
Y: Seborrheic Keratoses and other Benign Tumors
Y: Atopic Dermatitis Photos


In [102]:
X_train = np.array(X_train)

In [103]:
y_train.shape

(15557, 23)

## Examine data

In [68]:
import matplotlib.pyplot as plt

## Build Model

In [111]:
model_base = VGG16(weights='imagenet', include_top=False)

In [113]:
x = model_base.output
x = keras.Sequential([
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(23, activation='softmax')
    ])(x)
for layer in model_base.layers:
    layer.trainable = False

In [114]:
predictions = x
model = keras.models.Model(inputs = model_base.input, outputs = predictions)

In [115]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0   

In [116]:
num_classes = len(diseases)
label = keras.utils.to_categorical(y_train, num_classes)

In [117]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 5)

In [81]:
model.save('skin23class.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [75]:
X_test, y_test = labellingImg("/content/test/")

partScabies Lyme Disease and other Infestations and Bites
partEczema Photos
partWarts Molluscum and other Viral Infections
partNail Fungus and other Nail Disease
partActinic Keratosis Basal Cell Carcinoma and other Malignant Lesions
partHair Loss Photos Alopecia and other Hair Diseases
partBullous Disease Photos
partVasculitis Photos
partExanthems and Drug Eruptions
partLupus and other Connective Tissue diseases
partPsoriasis pictures Lichen Planus and related diseases
partUrticaria Hives
partPoison Ivy Photos and other Contact Dermatitis
partVascular Tumors
partSystemic Disease
partMelanoma Skin Cancer Nevi and Moles
partHerpes HPV and other STDs Photos
partAcne and Rosacea Photos
partLight Diseases and Disorders of Pigmentation
partTinea Ringworm Candidiasis and other Fungal Infections
partCellulitis Impetigo and other Bacterial Infections
partSeborrheic Keratoses and other Benign Tumors
partAtopic Dermatitis Photos


In [76]:
num_classes = len(os.listdir('/content/test/'))
y_test = keras.utils.to_categorical(np.array(y_test), num_classes)
X_test = np.array(X_test)

In [84]:
evaluation_metrics = model.evaluate(X_test, test)

# Access accuracy and loss
accuracy = evaluation_metrics[1]
loss = evaluation_metrics[0]

# Print the evaluation results
print(f'Accuracy: {accuracy}')
print(f'Loss: {loss}')


487/487 [==============================] - 47s 97ms/step - loss: 0.2735 - accuracy: 0.9169
Accuracy: 0.9168862700462341
Loss: 0.2735297679901123
